<a href="https://colab.research.google.com/github/1moi6/pyfuzzy-toolbox/blob/main/notebooks_colab/04_dynamics/04_pfuzzy_continuous_population.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema p-Fuzzy Contínuo: População Única

Modelo de crescimento populacional usando regras fuzzy.

**Modelo contínuo:**
$$\frac{dx}{dt} = f(x)$$

onde $f(x)$ é determinado por regras fuzzy baseadas na população atual.

In [ ]:
!pip install pyfuzzy-toolbox matplotlib numpy -q

In [ ]:
import fuzzy_systems as fs
import numpy as np
import matplotlib.pyplot as plt

## Sistema de Inferência Fuzzy

Regras baseadas em crescimento logístico:
- População baixa → crescimento forte
- População média → crescimento moderado
- População alta → declínio (competição/recursos)

In [ ]:
fis = fs.MamdaniSystem(name="Population")

# Entrada: população atual
fis.add_input('population', (0, 100))
fis.add_term('population', 'baixa', 'gaussian', (10, 10))
fis.add_term('population', 'media', 'gaussian', (50, 12))
fis.add_term('population', 'alta', 'gaussian', (90, 10))

# Saída: taxa de variação da população (dx/dt)
fis.add_output('d_population', (-5, 5))
fis.add_term('d_population', 'declinio', 'triangular', (-5, -2.5, 0))
fis.add_term('d_population', 'estavel', 'triangular', (-1, 0, 1))
fis.add_term('d_population', 'crescimento', 'triangular', (0, 2.5, 5))

# Regras (sintaxe de tupla plana)
fis.add_rules([
    ('baixa', 'crescimento'),
    ('media', 'estavel'),
    ('alta', 'declinio')
])

## Simulação

In [ ]:
pfuzzy = fs.dynamics.PFuzzyContinuous(fis=fis, mode='absolute', method='rk4')

# Diferentes condições iniciais
initial_conditions = [10, 30, 50, 70, 90]
colors = ['blue', 'green', 'orange', 'red', 'purple']

fig, ax = plt.subplots(figsize=(12, 6))

for x0, color in zip(initial_conditions, colors):
    traj = pfuzzy.simulate(x0={'population': x0}, t_span=(0, 20), dt=0.01)
    ax.plot(pfuzzy.time, traj[:, 0], color=color, linewidth=2, label=f'x₀={x0}')

ax.set_xlabel('Tempo', fontsize=12)
ax.set_ylabel('População', fontsize=12)
ax.set_title('Crescimento Populacional p-Fuzzy Contínuo (RK4)', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()